### Application of the Noisy CR Gate in a Quantum Circuit

In superconducting quantum processors (e.g., IBM devices), two-qubit gates like the **cross-resonance (CR) gate** are central to generating entanglement. However, physical noise during gate execution alters the ideal unitary evolution. To simulate this, we apply a **noisy two-qubit operator** instead of a perfect one.

---

#### The Ideal Model

The ideal CR gate can be thought of as a controlled rotation in the $ XY $-plane:

$$
U = \exp\left( -i \frac{\theta}{2} R_{XY}(\phi) \right)
$$

In a noiseless circuit, this unitary would be inserted directly into a quantum circuit acting on qubits $ q_0 $ (control) and $ q_1 $ (target):

$$
|\psi\rangle \rightarrow U |\psi\rangle
$$

---

#### The Noisy Substitution

Instead of applying $ U $, the simulator inserts a full noisy evolution operator:

$$
U_{\text{noisy}} = U \cdot \exp(D) \cdot \exp(i I)
$$

where:
- $ D = D_{\text{ctr}} + D_{\text{trg}} $: deterministic drift from T₁ relaxation,
- $ I = \sum_k I_k $: stochastic contributions from Itô noise (relaxation and depolarization).

This operator **is still a 4×4 matrix**, acting on the same two-qubit Hilbert space, so it can **replace** the ideal CR gate directly in a circuit simulation.

---

#### Simulation Usage

In a noisy circuit simulation, this gate would be inserted into the circuit just like any two-qubit gate, but with noise effects embedded:

1. The simulator initializes the system state $ \rho $ or $ |\psi\rangle $.
2. At the point where the CR gate is scheduled, the simulator computes $ U_{\text{noisy}} $ with sampled Itô integrals.
3. The gate is applied via matrix multiplication:
   - For statevector simulation:
     $$
     |\psi\rangle \rightarrow U_{\text{noisy}} |\psi\rangle
     $$
   - For density matrix simulation:
     $$
     \rho \rightarrow U_{\text{noisy}} \rho U_{\text{noisy}}^\dagger
     $$

Because $ U_{\text{noisy}} $ is non-unitary (due to decoherence), repeated applications across the circuit will accumulate realistic error.

---

#### Summary

The two-qubit noise model acts as a **replacement operator** for an ideal gate. It incorporates both:
- **Deterministic effects** (e.g., T₁ drift),
- **Stochastic quantum noise** (e.g., depolarizing and relaxation modeled by Itô integrals).

This operator is inserted at the point in the circuit where the CR gate would normally act, ensuring that simulations capture both coherent logic and physical decoherence.


### Noisy Cross-Resonance (CR) Gate — Initialization and Unitary Evolution

This section models the **Cross-Resonance (CR)** two-qubit gate with both **depolarizing** and **relaxation** noise using second-order Itô stochastic approximations. The model accounts for noise acting separately on the **control** and **target** qubits.

---

#### Parameters and Rescaling

Let:

- $ \theta $: rotation angle on the Bloch sphere
- $ \phi $: drive phase defining the rotation axis
- $ t_{\text{CR}} $: duration of the CR gate
- $ p_{\text{CR}} $: depolarizing error probability
- $ T_1, T_2 $: amplitude damping and dephasing times

We define a normalized time scale:

$$
t_g = 35 \, \text{ns}
\qquad a = \frac{t_{\text{CR}}}{t_g}
\qquad \omega = \theta
$$

The depolarizing error scale is computed as:

$$
e_d^{\text{CR}} = \sqrt{ \frac{p_{\text{CR}}}{4a} }
$$

For relaxation, define rates:

$$
e_1 = \sqrt{ \frac{t_g}{T_1} }
\qquad
e_2 = \sqrt{ \frac{t_g}{T_2} }
\qquad
e_p = \sqrt{ \frac{1}{2} \left( e_2^2 - \frac{e_1^2}{2} \right) }
$$

These quantities are computed separately for control (`ctr`) and target (`trg`) qubits.

---

#### 🔁 Unitary Evolution

The noiseless CR unitary is given in the computational basis (4×4 matrix):

$$
U_{\text{CR}} = \exp\left( -i \frac{\theta}{2} R_{XY}(\phi) \right)
$$

The matrix form is:

$$
U = \begin{bmatrix}
\cos\frac{\theta}{2} & -i \sin\frac{\theta}{2} e^{-i\phi} & 0 & 0 \\
-i \sin\frac{\theta}{2} e^{i\phi} & \cos\frac{\theta}{2} & 0 & 0 \\
0 & 0 & \cos\frac{\theta}{2} & i \sin\frac{\theta}{2} e^{-i\phi} \\
0 & 0 & i \sin\frac{\theta}{2} e^{i\phi} & \cos\frac{\theta}{2}
\end{bmatrix}
$$

This corresponds to a conditional rotation of the target qubit, depending on the state of the control qubit.


In [1]:
import numpy as np
from numpy import array, exp, sqrt, pi, sin, cos
from numpy.random import multivariate_normal, normal
from scipy.linalg import expm

In [2]:
def cr_gate_parameters(theta, t_cr, p_cr, T1_ctr, T2_ctr, T1_trg, T2_trg):
    """
    Computes time scaling, depolarizing strength, and noise rates for the CR gate.
    
    Returns:
        tg, omega, a, ed_cr, e1_ctr, ep_ctr, e1_trg, ep_trg
    """
    tg = 35e-9  # Reference gate time in seconds
    omega = theta
    a = t_cr / tg
    ed_cr = sqrt(p_cr / (4 * a))  # Depolarizing error scale

    # Control qubit: amplitude damping and dephasing
    e1_ctr = 0 if T1_ctr == 0 else sqrt(tg / T1_ctr)
    if T2_ctr == 0:
        ep_ctr = 0
    else:
        e2_ctr = sqrt(tg / T2_ctr)
        ep_ctr = sqrt(0.5 * (e2_ctr**2 - 0.5 * e1_ctr**2))

    # Target qubit: amplitude damping and dephasing
    e1_trg = 0 if T1_trg == 0 else sqrt(tg / T1_trg)
    if T2_trg == 0:
        ep_trg = 0
    else:
        e2_trg = sqrt(tg / T2_trg)
        ep_trg = sqrt(0.5 * (e2_trg**2 - 0.5 * e1_trg**2))

    return tg, omega, a, ed_cr, e1_ctr, ep_ctr, e1_trg, ep_trg



def cr_unitary(theta, phi):
    """
    Constructs the noiseless CR unitary (4x4 matrix).
    """
    return np.array([
        [np.cos(theta/2), -1j * np.sin(theta/2) * np.exp(-1j * phi), 0, 0],
        [-1j * np.sin(theta/2) * np.exp(1j * phi), np.cos(theta/2), 0, 0],
        [0, 0, np.cos(theta/2), 1j * np.sin(theta/2) * np.exp(-1j * phi)],
        [0, 0, 1j * np.sin(theta/2) * np.exp(1j * phi), np.cos(theta/2)]
    ])



### 🧊 Relaxation Noise — Control Qubit (σ₋ ⊗ I Channel)

This section models **amplitude damping** noise acting on the **control qubit**, represented by the operator:

$$
\sigma_{-} \otimes I = 
\begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

The noise is simulated using a **stochastic process** with two correlated Gaussian integrals $ I_{r,1} $ and $ I_{r,2} $, derived from the Itô integral expansion. Their covariance matrix is built from the following integrals over time $ s \in [0,1] $:

- Variance of $ I_{r,1} $:
$$
V_{r}^{(1)} = \frac{1}{4\omega}(2a\omega + a\sin(2\omega))
$$

- Variance of $ I_{r,2} $:
$$
V_{r}^{(2)} = \frac{1}{4} \left( 2a - \frac{a \sin(2\omega)}{\omega} \right)
$$

- Covariance:
$$
\text{Cov}(I_{r,1}, I_{r,2}) = \frac{a \sin^2(\omega)}{2\omega}
$$

These values form a **2×2 Gaussian covariance matrix**, from which one sample is drawn to produce:

$$
I_r^{\text{ctr}} = e_1^{\text{ctr}} \cdot
\begin{bmatrix}
0 & 0 & I_{r,1} & i I_{r,2} e^{-i\phi} \\
0 & 0 & i I_{r,2} e^{i\phi} & I_{r,1} \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

This matrix represents the stochastic contribution of amplitude damping on the control qubit in the CR gate.


In [3]:
def control_relaxation_noise_matrix(a, omega, e1_ctr, phi):
    """
    Constructs the Itô noise matrix for the control qubit's relaxation channel (σ₋ ⊗ I)
    during the CR gate.

    Args:
        a (float): Normalized CR gate time (t_cr / tg)
        omega (float): Rotation frequency (typically θ)
        e1_ctr (float): Amplitude damping rate of the control qubit
        phi (float): Drive phase of the CR gate

    Returns:
        Ir_ctr (np.array): 4x4 stochastic noise matrix
    """

    # === Itô integrals' variances and covariances for σ₋ ⊗ I ===
    Vr_ctr_1 = (2*a*omega + a*np.sin(2*omega)) / (4*omega)
    Vr_ctr_2 = 0.25 * (2*a - (a * np.sin(2*omega)) / omega)
    Covr_ctr_12 = (a * (np.sin(omega)**2)) / (2*omega)

    # Mean and covariance matrix for Gaussian sampling
    meanr_ctr = [0, 0]
    covr_ctr = [[Vr_ctr_1, Covr_ctr_12], [Covr_ctr_12, Vr_ctr_2]]

    # Sample correlated Gaussian integrals
    sample_r_ctr = np.random.multivariate_normal(meanr_ctr, covr_ctr, 1)
    Ir_ctr_1 = sample_r_ctr[0, 0]
    Ir_ctr_2 = sample_r_ctr[0, 1]

    # === Assemble stochastic noise matrix ===
    Ir_ctr = e1_ctr * np.array([
        [0, 0, Ir_ctr_1, 1j * Ir_ctr_2 * np.exp(-1j * phi)],
        [0, 0, 1j * Ir_ctr_2 * np.exp(1j * phi), Ir_ctr_1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    return Ir_ctr


### 🧊 Relaxation Noise — Target Qubit (I ⊗ σ₋ Channel)

This section simulates **relaxation noise** acting on the **target qubit** during the CR gate via the operator:

$$
I \otimes \sigma_{-} =
\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
\end{bmatrix}
$$

A second-order Itô stochastic expansion is used. Three stochastic integrals are sampled:  
$ I_{r,1} $, $ I_{r,2} $, and $ W_r $ (with variance $ a $). Their contributions are scaled by the amplitude damping strength $ e_1^{\text{trg}} $.

---

#### 📐 Variance/Covariance Matrix

The Itô variances and covariances are:

- Variance of $ I_{r,1} $:

$$
V_{r}^{(1)} = \frac{1}{4} \left( 2a - \frac{a \sin(2\omega)}{\omega} \right)
$$

- Variance of $ I_{r,2} $:

$$
V_{r}^{(2)} = \frac{a}{16\omega} \left(6\omega - 8\sin(\omega) + \sin(2\omega)\right)
$$

- Covariance:

$$
\text{Cov}(I_{r,1}, I_{r,2}) = \frac{a}{\omega} \sin^4\left(\frac{\omega}{2}\right)
$$

- Covariances with Wiener process $ W_r $:

$$
\text{Cov}(I_{r,1}, W_r) = \frac{a}{\omega}(1 - \cos(\omega))
\qquad
\text{Cov}(I_{r,2}, W_r)


In [4]:
def target_relaxation_noise_matrix(a, omega, e1_trg, phi):
    """
    Constructs the Itô noise matrix for the target qubit's relaxation channel (I ⊗ σ₋)
    during the CR gate.

    Args:
        a (float): Normalized CR gate time (t_cr / tg)
        omega (float): Rotation frequency (typically θ)
        e1_trg (float): Amplitude damping rate of the target qubit
        phi (float): Drive phase of the CR gate

    Returns:
        Ir_trg (np.array): 4x4 stochastic noise matrix
    """

    # === Itô variances and covariances for I ⊗ σ₋ ===
    Vr_trg_1 = 0.25 * (2*a - (a * np.sin(2*omega)) / omega)
    Vr_trg_2 = (a / (16 * omega)) * (6*omega - 8*np.sin(omega) + np.sin(2*omega))
    Covr_trg_12 = (a / omega) * (np.sin(omega/2)**4)
    Covr_trg_1Wr = (a / omega) * (1 - np.cos(omega))
    Covr_trg_2Wr = (a / (2 * omega)) * (omega - np.sin(omega))

    # 3x3 covariance matrix for I_r1, I_r2, and Wr
    meanr_trg = [0, 0, 0]
    covr_trg = [
        [Vr_trg_1, Covr_trg_12, Covr_trg_1Wr],
        [Covr_trg_12, Vr_trg_2, Covr_trg_2Wr],
        [Covr_trg_1Wr, Covr_trg_2Wr, a]
    ]

    # Sample the stochastic integrals
    sample_r_trg = np.random.multivariate_normal(meanr_trg, covr_trg, 1)
    Ir_trg_1 = sample_r_trg[0, 0]
    Ir_trg_2 = sample_r_trg[0, 1]
    Wr_trg = sample_r_trg[0, 2]

    # === Build the relaxation noise matrix ===
    Ir_trg = e1_trg * np.array([
        [-1j * 0.5 * Ir_trg_1 * np.exp(1j * phi), Wr_trg - Ir_trg_2, 0, 0],
        [Ir_trg_2 * np.exp(2j * phi), 1j * 0.5 * Ir_trg_1 * np.exp(1j * phi), 0, 0],
        [0, 0, 1j * 0.5 * Ir_trg_1 * np.exp(1j * phi), Wr_trg - Ir_trg_2],
        [0, 0, Ir_trg_2 * np.exp(2j * phi), -1j * 0.5 * Ir_trg_1 * np.exp(1j * phi)]
    ])

    return Ir_trg


###  Z-Axis Depolarization Noise (Control and Target Qubits)

This section models the **Z-component depolarization** acting separately on the **control** and **target** qubits. This noise arises from fluctuations around the $ Z $-axis and is modeled using either a single Wiener process (for control) or a second-order correlated Itô process (for target).

---

####  Control Qubit — [Z ⊗ I]

For the control qubit, the depolarization process is modeled by a single Wiener process $ W_p^{\text{ctr}} \sim \mathcal{N}(0, a) $. The corresponding 4×4 matrix is:

$$
I_p^{\text{ctr}} = e_p^{\text{ctr}} \cdot
\begin{bmatrix}
W_p & 0 & 0 & 0 \\
0 & W_p & 0 & 0 \\
0 & 0 & -W_p & 0 \\
0 & 0 & 0 & -W_p \\
\end{bmatrix}
$$

---

####  Target Qubit — [I ⊗ Z]

For the target qubit, two correlated Gaussian integrals $ I_{p,1} $, $ I_{p,2} $ are drawn from a 2D Itô covariance matrix:

- Variance of $ I_{p,1} $:
$$
V_p^{(1)} = \frac{1}{4\omega}(2a\omega + a\sin(2\omega))
$$

- Variance of $ I_{p,2} $:
$$
V_p^{(2)} = \frac{1}{4} \left( 2a - \frac{a \sin(2\omega)}{\omega} \right)
$$

- Covariance:
$$
\text{Cov}(I_{p,1}, I_{p,2}) = \frac{a \sin^2(\omega)}{2\omega}
$$

The resulting matrix is:

$$
I_p^{\text{trg}} = e_p^{\text{trg}} \cdot
\begin{bmatrix}
I_{p,1} & -i I_{p,2} e^{-i\phi} & 0 & 0 \\
i I_{p,2} e^{i\phi} & -I_{p,1} & 0 & 0 \\
0 & 0 & I_{p,1} & i I_{p,2} e^{-i\phi} \\
0 & 0 & -i I_{p,2} e^{i\phi} & -I_{p,1} \\
\end{bmatrix}
$$


In [5]:
def z_axis_depolarizing_noise(a, omega, ep_ctr, ep_trg, phi):
    """
    Constructs depolarization noise matrices for control (Z ⊗ I) and target (I ⊗ Z) qubits
    due to stochastic fluctuations along the Z-axis.

    Args:
        a (float): Normalized gate time (t_cr / tg)
        omega (float): Rotation angle/frequency
        ep_ctr (float): Control qubit pure dephasing rate
        ep_trg (float): Target qubit pure dephasing rate
        phi (float): Drive phase

    Returns:
        Ip_ctr, Ip_trg (tuple of np.array): Two 4x4 depolarizing noise matrices
    """

    # === CONTROL QUBIT: Z ⊗ I ===
    Wp_ctr = np.random.normal(0, np.sqrt(a))
    Ip_ctr = ep_ctr * np.array([
        [Wp_ctr, 0, 0, 0],
        [0, Wp_ctr, 0, 0],
        [0, 0, -Wp_ctr, 0],
        [0, 0, 0, -Wp_ctr]
    ])

    # === TARGET QUBIT: I ⊗ Z ===
    Vp_trg_1 = (2*a*omega + a*np.sin(2*omega)) / (4*omega)
    Vp_trg_2 = 0.25 * (2*a - (a*np.sin(2*omega))/omega)
    Covp_trg_12 = (a * (np.sin(omega)**2)) / (2*omega)

    meanp_trg = [0, 0]
    covp_trg = [[Vp_trg_1, Covp_trg_12], [Covp_trg_12, Vp_trg_2]]

    sample_p_trg = np.random.multivariate_normal(meanp_trg, covp_trg, 1)
    Ip_trg_1 = sample_p_trg[0, 0]
    Ip_trg_2 = sample_p_trg[0, 1]

    Ip_trg = ep_trg * np.array([
        [Ip_trg_1, -1j * Ip_trg_2 * np.exp(-1j * phi), 0, 0],
        [1j * Ip_trg_2 * np.exp(1j * phi), -Ip_trg_1, 0, 0],
        [0, 0, Ip_trg_1, 1j * Ip_trg_2 * np.exp(-1j * phi)],
        [0, 0, -1j * Ip_trg_2 * np.exp(1j * phi), -Ip_trg_1]
    ])

    return Ip_ctr, Ip_trg


### Deterministic Contribution from Relaxation (T₁ Noise)

In addition to stochastic contributions, the CR gate evolution also includes a **deterministic drift** due to amplitude damping (T₁ relaxation), captured by first-order perturbative integration over the Lindblad terms.

This contribution is derived from integrating the dissipator of the amplitude damping channel and scales quadratically with the damping rate $ e_1^2 $.

---

#### 🔹 Control Qubit (Z ⊗ I)

Relaxation causes the excited state population to decay over time. For the control qubit, this decay is modeled by applying a constant damping to the subspace $|10\rangle$, $|11\rangle$, giving:

$$
D_{\text{ctr}} = -\frac{e_1^2}{2} \cdot
\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & a & 0 \\
0 & 0 & 0 & a \\
\end{bmatrix}
$$

This matrix damps the excited state probability in a constant way proportional to the gate duration $ a = t_{\text{CR}} / t_g $.

---

#### 🔸 Target Qubit (I ⊗ σ₋)

For the target qubit, the deterministic drift involves off-diagonal elements due to coupling in the rotating frame. The matrix is defined by three integrals:

- Diagonal decay terms:
$$
\text{det}_1 = \frac{a\omega - a \sin(\omega)}{2\omega}
\quad
\text{det}_3 = \frac{a}{2\omega}(\omega + \sin(\omega))
$$

- Off-diagonal contributions:
$$
\text{det}_2 = \frac{a}{\omega}(1 - \cos(\omega))
$$

The deterministic matrix is:

$$
D_{\text{trg}} = -\frac{e_1^2}{2} \cdot
\begin{bmatrix}
\text{det}_1 & \frac{i}{2} \text{det}_2 e^{-i\phi} & 0 & 0 \\
-\frac{i}{2} \text{det}_2 e^{i\phi} & \text{det}_3 & 0 & 0 \\
0 & 0 & \text{det}_1 & -\frac{i}{2} \text{det}_2 e^{-i\phi} \\
0 & 0 & \frac{i}{2} \text{det}_2 e^{i\phi} & \text{det}_3 \\
\end{bmatrix}
$$

This deterministic drift is applied multiplicatively via the matrix exponential.


In [6]:
def relaxation_drift_matrices(a, omega, e1_ctr, e1_trg, phi):
    """
    Computes the deterministic drift matrices caused by relaxation (T1 noise)
    for both control and target qubits in the CR gate.

    Args:
        a (float): Normalized CR gate time (t_cr / tg)
        omega (float): Gate rotation frequency (theta)
        e1_ctr (float): Control qubit T1 damping rate
        e1_trg (float): Target qubit T1 damping rate
        phi (float): Drive phase

    Returns:
        deterministic_r_ctr, deterministic_r_trg (np.array): 4x4 drift matrices
    """

    # === Integrals for deterministic drift (target qubit only) ===
    det1 = (a * omega - a * np.sin(omega)) / (2 * omega)
    det2 = (a / omega) * (1 - np.cos(omega))
    det3 = (a / (2 * omega)) * (omega + np.sin(omega))

    # === Control qubit deterministic matrix: Z ⊗ I ===
    deterministic_r_ctr = -0.5 * e1_ctr**2 * np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, a, 0],
        [0, 0, 0, a]
    ])

    # === Target qubit deterministic matrix: I ⊗ σ₋ ===
    deterministic_r_trg = -0.5 * e1_trg**2 * np.array([
        [det1, 1j * 0.5 * det2 * np.exp(-1j * phi), 0, 0],
        [-1j * 0.5 * det2 * np.exp(1j * phi), det3, 0, 0],
        [0, 0, det1, -1j * 0.5 * det2 * np.exp(-1j * phi)],
        [0, 0, 1j * 0.5 * det2 * np.exp(1j * phi), det3]
    ])

    return deterministic_r_ctr, deterministic_r_trg


### Y-Axis Depolarization — Control Qubit (Y ⊗ I)

This section models depolarizing noise affecting the **control qubit** along the $ Y $-axis, represented by the operator:

$
Y \otimes I =
\begin{bmatrix}
0 & 0 & -i & 0 \\
0 & 0 & 0 & i \\
i & 0 & 0 & 0 \\
0 & -i & 0 & 0
\end{bmatrix}
$

---

####  Variance and Covariance Structure

To simulate stochastic contributions along this axis, two correlated Gaussian integrals $ I_{Y,1}, I_{Y,2} $ are drawn from a covariance matrix derived from:

- Variance of $ I_{Y,1} $:
$
V_Y^{(1)} = \frac{1}{4\omega}(2a\omega + a \sin(2\omega))
$

- Variance of $ I_{Y,2} $:
$
V_Y^{(2)} = \frac{1}{4} \left( 2a - \frac{a \sin(2\omega)}{\omega} \right)
$

- Covariance:
$
\text{Cov}(I_{Y,1}, I_{Y,2}) = \frac{a \sin^2(\omega)}{2\omega}
$

A 2D multivariate normal sample $(I_{Y,1}, I_{Y,2})$ is then scaled by the depolarizing rate $ e_d^{\text{CR}} $, and inserted into the 4×4 Itô noise matrix as:

$
I_Y^{\text{ctr}} = e_d^{\text{CR}} \cdot
\begin{bmatrix}
0 & 0 & -i I_{Y,1} & I_{Y,2} e^{-i\phi} \\
0 & 0 & I_{Y,2} e^{i\phi} & -i I_{Y,1} \\
i I_{Y,1} & I_{Y,2} e^{-i\phi} & 0 & 0 \\
I_{Y,2} e^{i\phi} & i I_{Y,1} & 0 & 0
\end{bmatrix}
$


In [7]:
def control_y_depolarizing_noise_matrix(a, omega, ed_cr, phi):
    """
    Constructs the Y-axis depolarization noise matrix for the control qubit (Y ⊗ I).

    Args:
        a (float): Normalized gate time (t_cr / tg)
        omega (float): Gate frequency (θ)
        ed_cr (float): Depolarizing error strength for CR gate
        phi (float): CR drive phase

    Returns:
        Idy_ctr (np.array): 4x4 depolarization noise matrix
    """

    # === Itô variances and covariances for Y ⊗ I ===
    Vdy_ctr_1 = (2*a*omega + a*np.sin(2*omega)) / (4*omega)
    Vdy_ctr_2 = 0.25 * (2*a - (a*np.sin(2*omega)) / omega)
    Covdy_ctr_12 = (a * (np.sin(omega)**2)) / (2*omega)

    # Construct covariance matrix for (I1, I2)
    meandy_ctr = [0, 0]
    covdy_ctr = [
        [Vdy_ctr_1, Covdy_ctr_12],
        [Covdy_ctr_12, Vdy_ctr_2]
    ]

    # Sample from 2D Gaussian
    sample_dy_ctr = np.random.multivariate_normal(meandy_ctr, covdy_ctr, 1)
    Idy_ctr_1 = sample_dy_ctr[0, 0]
    Idy_ctr_2 = sample_dy_ctr[0, 1]

    # === Build the noise matrix ===
    Idy_ctr = ed_cr * np.array([
        [0, 0, -1j * Idy_ctr_1, Idy_ctr_2 * np.exp(-1j * phi)],
        [0, 0, Idy_ctr_2 * np.exp(1j * phi), -1j * Idy_ctr_1],
        [1j * Idy_ctr_1, Idy_ctr_2 * np.exp(-1j * phi), 0, 0],
        [Idy_ctr_2 * np.exp(1j * phi), 1j * Idy_ctr_1, 0, 0]
    ])

    return Idy_ctr


### Z-Axis Depolarization — Control Qubit (Z ⊗ I)

This section models depolarizing noise acting along the **Z-axis** on the **control qubit**, represented by the operator:

$$
Z \otimes I =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & -1 & 0 \\
0 & 0 & 0 & -1 \\
\end{bmatrix}
$$

Unlike X and Y depolarizing channels (which involve two correlated Itô integrals), the Z-axis contribution is modeled by a **single Wiener process** $ W_z \sim \mathcal{N}(0, a) $.

The resulting stochastic noise matrix is:

$$
I_z^{\text{ctr}} = e_d^{\text{CR}} \cdot
\begin{bmatrix}
W_z & 0 & 0 & 0 \\
0 & W_z & 0 & 0 \\
0 & 0 & -W_z & 0 \\
0 & 0 & 0 & -W_z \\
\end{bmatrix}
$$

This reflects a random phase kick that flips sign between the $|0\rangle$ and $|1\rangle$ states of the control qubit.


In [8]:
def control_z_depolarizing_noise_matrix(a, ed_cr):
    """
    Constructs the Z-axis depolarization noise matrix for the control qubit (Z ⊗ I).

    Args:
        a (float): Normalized CR gate time (t_cr / tg)
        ed_cr (float): Depolarizing error strength for CR gate

    Returns:
        Idz_ctr (np.array): 4x4 Z depolarizing noise matrix
    """
    # Sample Wiener process with variance a
    Wdz_ctr = np.random.normal(0, np.sqrt(a))

    # Build noise matrix
    Idz_ctr = ed_cr * np.array([
        [Wdz_ctr, 0, 0, 0],
        [0, Wdz_ctr, 0, 0],
        [0, 0, -Wdz_ctr, 0],
        [0, 0, 0, -Wdz_ctr]
    ])

    return Idz_ctr


### X-Axis Depolarization — Target Qubit (I ⊗ X)

This section models depolarizing noise along the **X-axis** acting on the **target qubit**, represented by:

$$
I \otimes X =
\begin{bmatrix}
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$

This process includes three stochastic integrals $ I_{X,1}, I_{X,2}, W_X $ with the following statistical moments derived from the noise basis functions:

- Variance of $ I_{X,1} $:
$$
V_X^{(1)} = \frac{1}{4} \left( 2a - \frac{a \sin(2\omega)}{\omega} \right)
$$

- Variance of $ I_{X,2} $:
$$
V_X^{(2)} = \frac{a}{16\omega} \left(6\omega - 8\sin(\omega) + \sin(2\omega)\right)
$$

- Covariance:
$$
\text{Cov}(I_{X,1}, I_{X,2}) = \frac{a}{\omega} \sin^4\left(\frac{\omega}{2}\right)
$$

- Covariances with Wiener process $ W_X $:

$$
\text{Cov}(I_{X,1}, W_X) = \frac{a}{\omega}(1 - \cos(\omega)),
\quad
\text{Cov}(I_{X,2}, W_X) = \frac{a}{2\omega}(\omega - \sin(\omega))
$$

---

### Noise Matrix Form

Using a 3D Gaussian sample $(I_{X,1}, I_{X,2}, W_X)$, the depolarizing noise matrix is:

$$
I_X^{\text{trg}} = e_d^{\text{CR}} \cdot
\begin{bmatrix}
I_{X,1} \sin(\phi) & W_X + (\exp(-2i\phi) - 1) I_{X,2} & 0 & 0 \\
W_X + (\exp(2i\phi) - 1) I_{X,2} & -I_{X,1} \sin(\phi) & 0 & 0 \\
0 & 0 & -I_{X,1} \sin(\phi) & W_X + (\exp(-2i\phi) - 1) I_{X,2} \\
0 & 0 & W_X + (\exp(2i\phi) - 1) I_{X,2} & I_{X,1} \sin(\phi)
\end{bmatrix}
$$


In [9]:
def target_x_depolarizing_noise_matrix(a, omega, ed_cr, phi):
    """
    Constructs the X-axis depolarization noise matrix for the target qubit (I ⊗ X).

    Args:
        a (float): Normalized CR gate time (t_cr / tg)
        omega (float): Gate frequency (θ)
        ed_cr (float): Depolarizing error strength for CR gate
        phi (float): CR drive phase

    Returns:
        Idx_trg (np.array): 4x4 X depolarizing noise matrix
    """

    # === Variances and covariances for I ⊗ X ===
    Vdx_trg_1 = 0.25 * (2*a - (a * np.sin(2*omega)) / omega)
    Vdx_trg_2 = (a / (16 * omega)) * (6*omega - 8*np.sin(omega) + np.sin(2*omega))
    Covdx_trg_12 = (a / omega) * (np.sin(omega/2)**4)
    Covdx_trg_1Wdx = (a / omega) * (1 - np.cos(omega))
    Covdx_trg_2Wdx = (a / (2 * omega)) * (omega - np.sin(omega))

    # Mean and covariance for 3D Gaussian: [I1, I2, W]
    meandx_trg = np.array([0, 0, 0])
    covdx_trg = np.array([
        [Vdx_trg_1, Covdx_trg_12, Covdx_trg_1Wdx],
        [Covdx_trg_12, Vdx_trg_2, Covdx_trg_2Wdx],
        [Covdx_trg_1Wdx, Covdx_trg_2Wdx, a]
    ])

    # Sample the stochastic integrals
    sample_dx_trg = np.random.multivariate_normal(meandx_trg, covdx_trg, 1)
    Idx_trg_1 = sample_dx_trg[0, 0]
    Idx_trg_2 = sample_dx_trg[0, 1]
    Wdx_trg = sample_dx_trg[0, 2]

    # === Construct the noise matrix ===
    Idx_trg = ed_cr * np.array([
        [Idx_trg_1 * np.sin(phi), Wdx_trg + (np.exp(-2j * phi) - 1) * Idx_trg_2, 0, 0],
        [Wdx_trg + (np.exp(2j * phi) - 1) * Idx_trg_2, -Idx_trg_1 * np.sin(phi), 0, 0],
        [0, 0, -Idx_trg_1 * np.sin(phi), Wdx_trg + (np.exp(-2j * phi) - 1) * Idx_trg_2],
        [0, 0, Wdx_trg + (np.exp(2j * phi) - 1) * Idx_trg_2, Idx_trg_1 * np.sin(phi)]
    ])

    return Idx_trg


### Y-Axis Depolarization — Target Qubit (I ⊗ Y)

This section models depolarizing noise on the **target qubit** along the $ Y $-axis, using the operator:

$$
I \otimes Y =
\begin{bmatrix}
0 & -i & 0 & 0 \\
i & 0 & 0 & 0 \\
0 & 0 & 0 & -i \\
0 & 0 & i & 0
\end{bmatrix}
$$

As with the X-component, this model includes three Itô integrals $ I_{Y,1} $, $ I_{Y,2} $, and $ W_Y $ with covariances derived from trigonometric basis functions:

---

#### Variances and Covariances

- Variance of $ I_{Y,1} $:
$$
V_Y^{(1)} = \frac{1}{4} \left( 2a - \frac{a \sin(2\omega)}{\omega} \right)
$$

- Variance of $ I_{Y,2} $:
$$
V_Y^{(2)} = \frac{a}{16\omega} \left( 6\omega - 8\sin(\omega) + \sin(2\omega) \right)
$$

- Covariance:
$$
\text{Cov}(I_{Y,1}, I_{Y,2}) = \frac{a}{\omega} \sin^4\left( \frac{\omega}{2} \right)
$$

- Covariance with Wiener process:
$$
\text{Cov}(I_{Y,1}, W_Y) = \frac{a}{\omega} (1 - \cos(\omega)),
\quad
\text{Cov}(I_{Y,2}, W_Y) = \frac{a}{2\omega}(\omega - \sin(\omega))
$$

---

### Noise Matrix Form

Using the sampled integrals, the noise matrix becomes:

$$
I_Y^{\text{trg}} = e_d^{\text{CR}} \cdot
\begin{bmatrix}
- I_{Y,1} \cos(\phi) & -i W_Y + i(1 + e^{-2i\phi}) I_{Y,2} & 0 & 0 \\
i W_Y - i(1 + e^{2i\phi}) I_{Y,2} & I_{Y,1} \cos(\phi) & 0 & 0 \\
0 & 0 & I_{Y,1} \cos(\phi) & -i W_Y + i(1 + e^{-2i\phi}) I_{Y,2} \\
0 & 0 & i W_Y - i(1 + e^{2i\phi}) I_{Y,2} & - I_{Y,1} \cos(\phi)
\end{bmatrix}
$$


In [10]:
def target_y_depolarizing_noise_matrix(a, omega, ed_cr, phi):
    """
    Constructs the Y-axis depolarization noise matrix for the target qubit (I ⊗ Y).

    Args:
        a (float): Normalized CR gate time (t_cr / tg)
        omega (float): Gate frequency (θ)
        ed_cr (float): Depolarizing error strength for CR gate
        phi (float): CR drive phase

    Returns:
        Idy_trg (np.array): 4x4 Y depolarizing noise matrix
    """

    # === Variance/covariance calculations ===
    Vdy_trg_1 = 0.25 * (2*a - (a * np.sin(2*omega)) / omega)
    Vdy_trg_2 = (a / (16 * omega)) * (6*omega - 8*np.sin(omega) + np.sin(2*omega))
    Covdy_trg_12 = (a / omega) * (np.sin(omega/2)**4)
    Covdy_trg_1Wdy = (a / omega) * (1 - np.cos(omega))
    Covdy_trg_2Wdy = (a / (2 * omega)) * (omega - np.sin(omega))

    # 3D Gaussian: [I1, I2, W]
    meandy_trg = np.array([0, 0, 0])
    covdy_trg = np.array([
        [Vdy_trg_1, Covdy_trg_12, Covdy_trg_1Wdy],
        [Covdy_trg_12, Vdy_trg_2, Covdy_trg_2Wdy],
        [Covdy_trg_1Wdy, Covdy_trg_2Wdy, a]
    ])

    # Sample
    sample_dy_trg = np.random.multivariate_normal(meandy_trg, covdy_trg, 1)
    Idy_trg_1 = sample_dy_trg[0, 0]
    Idy_trg_2 = sample_dy_trg[0, 1]
    Wdy_trg = sample_dy_trg[0, 2]

    # === Construct matrix ===
    Idy_trg = ed_cr * np.array([
        [-Idy_trg_1 * np.cos(phi), -1j * Wdy_trg + 1j * (np.exp(-2j * phi) + 1) * Idy_trg_2, 0, 0],
        [1j * Wdy_trg - 1j * (np.exp(2j * phi) + 1) * Idy_trg_2, Idy_trg_1 * np.cos(phi), 0, 0],
        [0, 0, Idy_trg_1 * np.cos(phi), -1j * Wdy_trg + 1j * (np.exp(-2j * phi) + 1) * Idy_trg_2],
        [0, 0, 1j * Wdy_trg - 1j * (np.exp(2j * phi) + 1) * Idy_trg_2, -Idy_trg_1 * np.cos(phi)]
    ])

    return Idy_trg


###  Z-Axis Depolarization — Target Qubit (I ⊗ Z)

This section models depolarizing noise acting along the **Z-axis** of the **target qubit**, represented by the operator:

$$
I \otimes Z =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & -1 \\
\end{bmatrix}
$$

This contribution is modeled using a **2D Gaussian process** $ (I_{Z,1}, I_{Z,2}) $, with covariance structure derived from basis functions evaluated during the CR gate’s unitary evolution.

---

####  Variances and Covariances

- Variance of $ I_{Z,1} $:
$$
V_Z^{(1)} = \frac{1}{4\omega}(2a\omega + a \sin(2\omega))
$$

- Variance of $ I_{Z,2} $:
$$
V_Z^{(2)} = \frac{1}{4} \left( 2a - \frac{a \sin(2\omega)}{\omega} \right)
$$

- Covariance:
$$
\text{Cov}(I_{Z,1}, I_{Z,2}) = \frac{a \sin^2(\omega)}{2\omega}
$$

---

#### Noise Matrix Form

The resulting stochastic noise matrix is:

$$
I_Z^{\text{trg}} = e_d^{\text{CR}} \cdot
\begin{bmatrix}
I_{Z,1} & -i I_{Z,2} e^{-i\phi} & 0 & 0 \\
i I_{Z,2} e^{i\phi} & -I_{Z,1} & 0 & 0 \\
0 & 0 & I_{Z,1} & i I_{Z,2} e^{-i\phi} \\
0 & 0 & -i I_{Z,2} e^{i\phi} & -I_{Z,1}
\end{bmatrix}
$$


In [11]:
def target_z_depolarizing_noise_matrix(a, omega, ed_cr, phi):
    """
    Constructs the Z-axis depolarization noise matrix for the target qubit (I ⊗ Z).

    Args:
        a (float): Normalized CR gate time (t_cr / tg)
        omega (float): Gate frequency (θ)
        ed_cr (float): Depolarizing error strength for CR gate
        phi (float): CR drive phase

    Returns:
        Idz_trg (np.array): 4x4 Z depolarizing noise matrix
    """

    # === Variance and covariance calculations ===
    Vdz_trg_1 = (2*a*omega + a*np.sin(2*omega)) / (4*omega)
    Vdz_trg_2 = 0.25 * (2*a - (a * np.sin(2*omega)) / omega)
    Covdz_trg_12 = (a * (np.sin(omega)**2)) / (2*omega)

    # Construct the covariance matrix
    meandz_trg = [0, 0]
    covdz_trg = [
        [Vdz_trg_1, Covdz_trg_12],
        [Covdz_trg_12, Vdz_trg_2]
    ]

    # Sample the Gaussian integrals
    sample_dz_trg = np.random.multivariate_normal(meandz_trg, covdz_trg, 1)
    Idz_trg_1 = sample_dz_trg[0, 0]
    Idz_trg_2 = sample_dz_trg[0, 1]

    # === Build the matrix ===
    Idz_trg = ed_cr * np.array([
        [Idz_trg_1, -1j * Idz_trg_2 * np.exp(-1j * phi), 0, 0],
        [1j * Idz_trg_2 * np.exp(1j * phi), -Idz_trg_1, 0, 0],
        [0, 0, Idz_trg_1, 1j * Idz_trg_2 * np.exp(-1j * phi)],
        [0, 0, -1j * Idz_trg_2 * np.exp(1j * phi), -Idz_trg_1]
    ])

    return Idz_trg


### Noisy Cross-Resonance (CR) Gate Construction

We construct the noisy two-qubit CR gate as a product of three components:

$$
U_{\text{noisy}} = U \cdot \exp(D_{\text{ctr}} + D_{\text{trg}}) \cdot \exp\left( i \cdot I_{\text{total}} \right)
$$

---

#### Ideal Unitary Evolution

The noiseless CR unitary gate is:

$$
U = \exp\left( -i \frac{\theta}{2} R_{XY}(\phi) \right)
$$

This defines the ideal logical operation without noise.

---

#### Deterministic Drift from Relaxation

The deterministic non-unitary contribution from relaxation is given by:

$$
D_{\text{total}} = D_{\text{ctr}} + D_{\text{trg}}
$$

$$
\exp(D_{\text{total}}) = \exp(D_{\text{ctr}} + D_{\text{trg}})
$$

where $ D_{\text{ctr}} $, $ D_{\text{trg}} \in \mathbb{C}^{4 \times 4} $ are the drift matrices associated with amplitude damping (T₁ relaxation) on the control and target qubits.

---

#### Stochastic Noise from Itô Expansion

The stochastic contribution is modeled via second-order Itô processes. Let the total stochastic term be:

$$
\begin{aligned}
I_{\text{total}} =\ 
& I_{r}^{(\text{ctr})} + I_{r}^{(\text{trg})} +
  I_{p}^{(\text{ctr})} + I_{p}^{(\text{trg})} + \\
& I_{x}^{(\text{ctr})} + I_{y}^{(\text{ctr})} + I_{z}^{(\text{ctr})} + 
  I_{x}^{(\text{trg})} + I_{y}^{(\text{trg})} + I_{z}^{(\text{trg})}
\end{aligned}
$$

The stochastic evolution is then applied via:

$$
\exp\left( i \cdot I_{\text{total}} \right)
$$

---

#### Full Expression

Putting all parts together, the final noisy gate is:

$$
U_{\text{noisy}} = 
U \cdot 
\exp\left( D_{\text{ctr}} + D_{\text{trg}} \right) \cdot 
\exp\left( i \left(
  I_{r}^{(\text{ctr})} + I_{r}^{(\text{trg})} +
  I_{p}^{(\text{ctr})} + I_{p}^{(\text{trg})} +
  I_{x}^{(\text{ctr})} + I_{y}^{(\text{ctr})} + I_{z}^{(\text{ctr})} +
  I_{x}^{(\text{trg})} + I_{y}^{(\text{trg})} + I_{z}^{(\text{trg})}
\right) \right)
$$


In [12]:
def compose_noisy_cr_gate(U,
                          deterministic_r_ctr, deterministic_r_trg,
                          Ir_ctr, Ir_trg,
                          Ip_ctr, Ip_trg,
                          Idx_ctr, Idy_ctr, Idz_ctr,
                          Idx_trg, Idy_trg, Idz_trg):
    """
    Constructs the final noisy CR gate by combining all components.

    Args:
        U (np.array): Ideal CR unitary matrix
        deterministic_r_ctr (np.array): Drift from T1 (control)
        deterministic_r_trg (np.array): Drift from T1 (target)
        Ir_ctr, Ir_trg (np.array): Relaxation noise matrices
        Ip_ctr, Ip_trg (np.array): Z depolarizing noise
        Idx_ctr, Idy_ctr, Idz_ctr (np.array): Pauli X/Y/Z depolarization (control)
        Idx_trg, Idy_trg, Idz_trg (np.array): Pauli X/Y/Z depolarization (target)

    Returns:
        result (np.array): Final 4x4 noisy gate matrix
    """
    from scipy.linalg import expm

    # Deterministic relaxation contribution (first-order drift)
    deterministic_exp = expm(deterministic_r_ctr + deterministic_r_trg)

    # Stochastic Itô second-order noise (coherent exponential)
    stochastic_exp = expm(1j * (
        Ir_ctr + Ir_trg +
        Ip_ctr + Ip_trg +
        Idx_ctr + Idy_ctr + Idz_ctr +
        Idx_trg + Idy_trg + Idz_trg
    ))

    # Combine all components
    result = U @ deterministic_exp @ stochastic_exp
    return result
